In [2]:
import statsmodels.api as sm
import numpy as np
import pandas as pd

import pdb
import os
import matplotlib.pyplot as plt
import seaborn as sns

curr_dir = 'C:/Users/vayze/Desktop/GitHub_Repos/ma-mri'
curr_dir = '/home/vayzenbe/GitHub_Repos/docnet/'

In [64]:
rois = ['lLO_toolloc', 'lPFS_toolloc', 'rLO_toolloc', 'rPFS_toolloc','rPPC_spaceloc', 'rAPC_spaceloc']
rois = ['PPC_spaceloc', 'APC_spaceloc', 'LO_toolloc', 'PFS_toolloc']
roi_names = ['PPC', 'APC','LO', 'PFS']

models = ['skel', 'cornet_s', 'gbj']

rdms = pd.read_csv(f'{curr_dir}/modelling/rdms/all_rdms.csv')

rdms = (rdms-rdms.mean())/rdms.std()
                
iter_num = 10000

/home/vayzenbe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  if __name__ == '__main__':


In [65]:
def calc_rsa(roi, model_data, covs):
    # Pull out the data
    
    X = model_data[covs]
    y = roi
    model = sm.OLS(y, X).fit()

    
    return model.params, model.pvalues

In [71]:
lr = 'r'
rr = 'APC_spaceloc'
summary_df = pd.DataFrame(columns = ['roi'] + models + [s + '_se' for s in models] )
for rr in rois:
    for lr in ['l', 'r']:
        mean_beta,p_val = calc_rsa(rdms[f'{lr}{rr}'],rdms,models)
        
        beta_samples = []
        for itn in range(0,iter_num):
            temp_rdm = rdms[f'{lr}{rr}'].sample(frac = 1, replace = True, ignore_index = True)
            temp_beta, temp_p = calc_rsa(temp_rdm,rdms,models)
            beta_samples.append(temp_beta.to_list())
        
        beta_samples = pd.DataFrame(np.array(beta_samples))
        
        roi_data = [f'{lr}{rr}'] + mean_beta.to_list() + beta_samples.std(axis = 0).to_list()
        
        summary_df = summary_df.append(pd.Series(roi_data, index =summary_df.columns), ignore_index = True )
        

summary_df.to_csv(f'/home/vayzenbe/GitHub_Repos/docnet/results/RSA_betas.csv', index = False)
                                        
        

In [72]:
summary_df

,roi,skel,cornet_s,gbj,skel_se,cornet_s_se,gbj_se
0,lPPC_spaceloc,0.024122,-0.096930,-0.029544,0.077192,0.078381,0.060292
1,rPPC_spaceloc,0.328781,-0.015079,0.039416,0.077043,0.077309,0.060888
2,lAPC_spaceloc,0.039272,-0.040063,-0.064643,0.077382,0.078485,0.060563
3,rAPC_spaceloc,0.141851,-0.069428,-0.002464,0.077422,0.078101,0.060563
4,lLO_toolloc,0.171314,0.135390,-0.004607,0.077425,0.077408,0.059637
5,rLO_toolloc,0.042252,0.129312,0.008301,0.077957,0.078770,0.060489
6,lPFS_toolloc,0.138296,-0.060032,0.022474,0.077467,0.078512,0.060969
7,rPFS_toolloc,-0.100237,0.140995,0.011056,0.076937,0.077742,0.060702


In [ ]:
beta_sample

In [49]:
beta_samples = pd.DataFrame(np.array(beta_samples))

In [55]:
roi_data = [f'{lr}{rr}'] + mean_beta.to_list() + beta_samples.sem(axis = 0).to_list()
roi_data

['lPPC_spaceloc',
 0.02412248540437869,
 -0.0969298193274901,
 -0.02954411521415691,
 0.020812338815115152,
 0.031539649088171244,
 0.01922661454536177]

In [59]:
summary_df = pd.DataFrame(columns = ['roi'] + models + [s + '_se' for s in models] )
summary_df = summary_df.append(pd.Series(roi_data, index =summary_df.columns), ignore_index = True )
summary_df

,roi,skel,cornet_s,gbj,skel_se,cornet_s_se,gbj_se
0,lPPC_spaceloc,0.024122,-0.09693,-0.029544,0.020812,0.03154,0.019227


In [57]:
summary_df

,0,cornet_s,cornet_s_se,gbj,gbj_se,roi,skel,skel_se
0,lPPC_spaceloc,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.024122,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-0.09693,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-0.029544,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.020812,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.03154,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0.019227,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
temp_rdm

108    0.951908
155    0.557901
152   -0.061268
265   -1.600217
71    -1.584540
         ...   
261   -2.053389
83     1.471191
92    -0.238132
73    -0.900139
147   -1.038192
Name: rAPC_spaceloc, Length: 300, dtype: float64